In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels
import statsmodels.api as sm
import statsmodels.formula.api as smf
from scipy import stats
import warnings;
from pysqldf import SQLDF
import pandasql as psql
from matplotlib.ticker import FuncFormatter
from sklearn.model_selection import KFold
import sklearn.ensemble as ske
import lightgbm as lgb
from pandas.api.types import is_string_dtype
from pandas.api.types import is_numeric_dtype
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from math import sqrt

warnings.filterwarnings('ignore')

In [2]:
train01 = pd.read_csv("C:\\Kaggle\\Cars\\Data\\TrnDataForLGB.csv")
test01 = pd.read_csv("C:\\Kaggle\\Cars\\Data\\TstDataForLGB.csv")

Remove_List = ["id","Price","Name","Lag_Price2_MIN","Lag_Price2_MAX",
               "Lag_Price3_MIN","Lag_Price3_MAX","Lag_Price","Lag_Price3","Engine_Group",
               "Power_Group","TrainTestInd","CarCompName","RateChng1","RateChng2","RateChng3",
               "Lag_Price4_MIN","Lag_Price4_MAX","Lag_Price4_MIN_BY_MAX"]
feature_names = list(set(list(train01.columns)) - set(Remove_List))

In [8]:
train01[feature_names].columns[train01[feature_names].isnull().any()]

Index(['Lag_Price4', 'Lag_Price2', 'New_Price'], dtype='object')

In [9]:
test01[feature_names].columns[test01[feature_names].isnull().any()]

Index(['Lag_Price4', 'Lag_Price2', 'New_Price'], dtype='object')

In [10]:
train01['Lag_Price4'] = train01['Lag_Price4'].fillna(value=0)
train01['Lag_Price2'] = train01['Lag_Price2'].fillna(value=0)
train01['New_Price'] = train01['New_Price'].fillna(value=0)

test01['Lag_Price4'] = test01['Lag_Price4'].fillna(value=0)
test01['Lag_Price2'] = test01['Lag_Price2'].fillna(value=0)
test01['New_Price'] = test01['New_Price'].fillna(value=0)

In [11]:
print(train01[feature_names].columns[train01[feature_names].isnull().any()])
print(test01[feature_names].columns[test01[feature_names].isnull().any()])

Index([], dtype='object')
Index([], dtype='object')


In [12]:
x_train02 = train01[feature_names]
y_train02 = train01['Price']
x_sub2 = test01[feature_names]

In [13]:
train_stats = x_train02.describe()
train_stats = train_stats.transpose()
train_stats

,count,mean,std,min,25%,50%,75%,max
Fuel_TypeElectric,6019.0,0.000332,0.018227,0.0,0.00,0.00,0.0,1.00
CompNameCarName_MOBILIO,6019.0,0.002658,0.051494,0.0,0.00,0.00,0.0,1.00
Kilometers_Driven,6019.0,58738.380296,91268.843206,171.0,34000.00,53000.00,73000.0,6500000.00
CompNameCarName_VENTO,6019.0,0.017777,0.132151,0.0,0.00,0.00,0.0,1.00
CompNameCarName_ECOSPORT,6019.0,0.015783,0.124647,0.0,0.00,0.00,0.0,1.00
CompName_LAND,6019.0,0.010301,0.100977,0.0,0.00,0.00,0.0,1.00
CompNameCarName_DZIRE,6019.0,0.029241,0.168495,0.0,0.00,0.00,0.0,1.00
CompNameCarName_ACCENT,6019.0,0.002991,0.054608,0.0,0.00,0.00,0.0,1.00
CompNameCarName_BRIO,6019.0,0.010135,0.100168,0.0,0.00,0.00,0.0,1.00
Fuel_TypePetrol,6019.0,0.456222,0.498121,0.0,0.00,0.00,1.0,1.00


In [14]:
def norm(x):
  return(x - train_stats['mean']) / train_stats['std']
normed_x_train02 = norm(x_train02)
normed_x_sub2 = norm(x_sub2)

In [15]:
normed_x_train02.reset_index(drop = True, inplace = True)
kf = KFold(n_splits = 5, shuffle = True, random_state = 100)
kf.get_n_splits(normed_x_train02)

5

In [34]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers.advanced_activations import PReLU
from keras import optimizers
from keras.callbacks import EarlyStopping
import math

def nn_model():
    model = Sequential()
    model.add(Dense(100, input_dim = normed_x_train02.shape[1], init = 'he_normal'))
    model.add(Activation('tanh'))
    model.add(Dropout(0.3))
    model.add(Dense(100, init = 'he_normal'))
    model.add(Activation('tanh'))
    model.add(Dropout(0.3))
    model.add(Dense(30, init = 'he_normal'))
    model.add(Activation('tanh'))
    model.add(Dropout(0.2))
    model.add(Dense(1, init = 'he_normal'))
    model.compile(loss = 'mean_squared_error', optimizer = 'Adam')
    return(model)

In [35]:
IterationNum = 1
for train_index, test_index in kf.split(normed_x_train02):
    print("Running CV Iteration Num :", IterationNum)
    MOD_DATA_2_TRAIN, MOD_DATA_2_TEST = train01.iloc[train_index], train01.iloc[test_index]
    X_TRAIN, X_TEST = normed_x_train02.iloc[train_index], normed_x_train02.iloc[test_index]
    Y_TRAIN, Y_TEST = y_train02[train_index], y_train02[test_index]
    
    model = nn_model()
    es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)
    fin_model = model.fit(X_TRAIN, Y_TRAIN, validation_data = (X_TEST, Y_TEST), epochs = 100, batch_size = 8, verbose = 1, callbacks=[es])
    
    MOD_DATA_2_TEST['Predicted_Model_Value'] = model.predict(X_TEST)
    if(IterationNum == 1):
        CV_SCORED_DATA = MOD_DATA_2_TEST.copy(deep=True)
        CV_SCORED_DATA.reset_index(drop = True, inplace = True)
        test_scored = model.predict(normed_x_sub2)
    else:
        CV_SCORED_DATA = pd.concat([CV_SCORED_DATA,MOD_DATA_2_TEST])
        CV_SCORED_DATA.reset_index(drop = True, inplace = True)
        test_scored = test_scored + model.predict(normed_x_sub2)
                    
    IterationNum = IterationNum + 1
    
    print("Test RMSE : ",sqrt(mean_squared_error(MOD_DATA_2_TEST["Price"], MOD_DATA_2_TEST['Predicted_Model_Value'])))
    
print("CV RMSE : ",sqrt(mean_squared_error(CV_SCORED_DATA["Price"], CV_SCORED_DATA['Predicted_Model_Value'])))
#CV RMSE :  0.17607317726030516

Running CV Iteration Num : 1
Train on 4815 samples, validate on 1204 samples
Epoch 1/100
4815/4815 [==============================] - 3s 520us/step - loss: 1.2029 - val_loss: 0.2436
Epoch 2/100
4815/4815 [==============================] - 1s 264us/step - loss: 0.3484 - val_loss: 0.1364
Epoch 3/100
4815/4815 [==============================] - 1s 269us/step - loss: 0.2443 - val_loss: 0.1142
Epoch 4/100
4815/4815 [==============================] - 1s 276us/step - loss: 0.1835 - val_loss: 0.1186
Epoch 5/100
4815/4815 [==============================] - 1s 305us/step - loss: 0.1519 - val_loss: 0.0863
Epoch 6/100
4815/4815 [==============================] - 1s 269us/step - loss: 0.1375 - val_loss: 0.0694
Epoch 7/100
4815/4815 [==============================] - 1s 264us/step - loss: 0.1273 - val_loss: 0.0663
Epoch 8/100
4815/4815 [==============================] - 1s 263us/step - loss: 0.1193 - val_loss: 0.0900
Epoch 9/100
4815/4815 [==============================] - 1s 273us/step - loss: 0.10

4815/4815 [==============================] - 1s 279us/step - loss: 0.0490 - val_loss: 0.0369
Epoch 28/100
4815/4815 [==============================] - 1s 254us/step - loss: 0.0465 - val_loss: 0.0369
Epoch 29/100
4815/4815 [==============================] - 1s 259us/step - loss: 0.0495 - val_loss: 0.0361
Epoch 30/100
4815/4815 [==============================] - 1s 272us/step - loss: 0.0470 - val_loss: 0.0361
Epoch 31/100
4815/4815 [==============================] - 1s 265us/step - loss: 0.0469 - val_loss: 0.0355
Epoch 32/100
4815/4815 [==============================] - 1s 274us/step - loss: 0.0449 - val_loss: 0.0344
Epoch 33/100
4815/4815 [==============================] - 1s 262us/step - loss: 0.0447 - val_loss: 0.0378
Epoch 34/100
4815/4815 [==============================] - 1s 268us/step - loss: 0.0433 - val_loss: 0.0345
Epoch 35/100
4815/4815 [==============================] - 1s 260us/step - loss: 0.0437 - val_loss: 0.0349
Epoch 36/100
4815/4815 [==============================] - 1

4815/4815 [==============================] - 1s 262us/step - loss: 0.0438 - val_loss: 0.0402
Epoch 37/100
4815/4815 [==============================] - 1s 260us/step - loss: 0.0416 - val_loss: 0.0399
Epoch 38/100
4815/4815 [==============================] - 1s 275us/step - loss: 0.0432 - val_loss: 0.0391
Epoch 39/100
4815/4815 [==============================] - 1s 276us/step - loss: 0.0442 - val_loss: 0.0377
Epoch 40/100
4815/4815 [==============================] - 1s 260us/step - loss: 0.0424 - val_loss: 0.0420
Epoch 41/100
4815/4815 [==============================] - 1s 253us/step - loss: 0.0426 - val_loss: 0.0395
Epoch 42/100
4815/4815 [==============================] - 1s 262us/step - loss: 0.0416 - val_loss: 0.0406
Epoch 43/100
4815/4815 [==============================] - 1s 264us/step - loss: 0.0418 - val_loss: 0.0418
Epoch 44/100
4815/4815 [==============================] - 1s 263us/step - loss: 0.0420 - val_loss: 0.0383
Epoch 45/100
4815/4815 [==============================] - 1

Epoch 2/100
4816/4816 [==============================] - 1s 270us/step - loss: 0.3779 - val_loss: 0.1650
Epoch 3/100
4816/4816 [==============================] - 1s 272us/step - loss: 0.2460 - val_loss: 0.1097
Epoch 4/100
4816/4816 [==============================] - 1s 284us/step - loss: 0.1842 - val_loss: 0.1077
Epoch 5/100
4816/4816 [==============================] - 1s 270us/step - loss: 0.1592 - val_loss: 0.0705
Epoch 6/100
4816/4816 [==============================] - 1s 260us/step - loss: 0.1416 - val_loss: 0.0732
Epoch 7/100
4816/4816 [==============================] - 1s 277us/step - loss: 0.1266 - val_loss: 0.0687
Epoch 8/100
4816/4816 [==============================] - 1s 304us/step - loss: 0.1176 - val_loss: 0.0532
Epoch 9/100
4816/4816 [==============================] - 1s 264us/step - loss: 0.1097 - val_loss: 0.0437
Epoch 10/100
4816/4816 [==============================] - 1s 285us/step - loss: 0.1038 - val_loss: 0.0416
Epoch 11/100
4816/4816 [==============================

In [36]:
from scipy import stats
test_scored2 = test_scored / 5.0
stats.describe(test_scored2)

DescribeResult(nobs=1234, minmax=(array([0.8876152], dtype=float32), array([3.7292836], dtype=float32)), mean=array([1.9737476], dtype=float32), variance=array([0.53152084], dtype=float32), skewness=array([0.7217], dtype=float32), kurtosis=array([-0.2060821], dtype=float32))

In [37]:
test_scored3 = pd.DataFrame({'Price' : np.exp(test_scored2[:,0])-1})
test_scored3.head()

,Price
0,2.477000
1,2.579717
2,18.617081
3,4.126414
4,4.404558


In [38]:
CV_SCORED_DATA.to_csv("C:\\Kaggle\\Cars\\CV_Scored\\20190716_Keras01_CVTRAIN_DS.csv",
                      index = False)
test_scored3.to_csv("C:\\Kaggle\\Cars\\Submission\\20190716_Keras01_TEST_DS.csv",
                    index = False)